In [7]:
import pathlib
import numpy as np
import scipy

Convert Neuronexus probe timestamps to Neuropixels probe timestamps using alignment data,
then write a fixed version of the DS_TYPE12 file.

In [55]:
PATH = pathlib.Path('/home/larend/data/dvorak-ca3')

def fix_NN_timestamps(ds_sample_NN, SYNC):
    sync_NN = SYNC['tsNN'].flatten() # NN sample index where sync pulse occurred
    sync_NP1 = SYNC['tsNP1'].flatten() # NP1 AP clock time when sync pulse occurred

    # trim to same length (may differ by 1)
    n_sync = min(len(sync_NN), len(sync_NP1))
    sync_NN = sync_NN[:n_sync]
    sync_NP1 = sync_NP1[:n_sync]
    
    ts_NP1 = SYNC['timestampsNP1'].flatten() # NP1 AP clock time when sample collected
    ts_NP1_LF = SYNC['timestampsNP_LP1'].flatten() # NP1 LF clock time when sample collected

    # convert 2000 Hz Neuronexus sample index to 30000 Hz Neuropixels AP sample time
    time_mask = (sync_NN[0] < ds_sample_NN) & (ds_sample_NN < sync_NN[-1])
    ds_sample_NN = ds_sample_NN[time_mask] # select DS within window where SYNC data exists
    ds_ts_NP1 = np.interp(ds_sample_NN, sync_NN, sync_NP1).astype('int') # NP1 AP clock time when DS occurred

    # convert 30000 Hz Neuropixels AP sample time to AP data array sample index
    sample_NP1 = np.arange(len(ts_NP1))
    ds_sample_NP1 = np.interp(ds_ts_NP1, ts_NP1, sample_NP1).astype('int') # NP1 AP sample index when DS occurred

    # convert AP sample index to Neuropixels LF (2500 Hz) sample index
    ds_sample_NP1_LF = ds_sample_NP1 // 12 # NP1 LF sample index when DS occurred
    return time_mask, ds_sample_NP1_LF

def fix_DS_TYPE12(time_mask, ds_sample_NP, DS_TYPE12):
    struct = DS_TYPE12.copy()
    struct['kType1inf'] = struct['kType1inf'][:, time_mask]
    struct['kType2inf'] = struct['kType2inf'][:, time_mask]
    struct['kType1sup'] = struct['kType1sup'][:, time_mask]
    struct['kType2sup'] = struct['kType2sup'][:, time_mask]
    struct['matches'] = struct['matches'][:, time_mask]
    struct['samplesDS'] = struct['samplesDS'][time_mask, :]
    struct['samplesDS'][:, 0] = ds_sample_NP
    return struct

In [56]:
session = '2020-01-16_17-35-48'
DS_TYPE_12_path = PATH / 'fix_NN_timestamps' / 'DS_TYPE12' / "2020-01-16_17-35-46_L700.mat"
DS_TYPE_12_path_fixed = PATH / session / f'{session}_DS_TYPE12.mat'
SYNC_path = PATH / 'fix_NN_timestamps' / 'SYNC' / "L700.mat"

DS_TYPE12 = scipy.io.loadmat(str(DS_TYPE_12_path))
ds_sample_NN = DS_TYPE12['samplesDS'].flatten()

SYNC = scipy.io.loadmat(SYNC_path)

time_mask, ds_sample_NP = fix_NN_timestamps(ds_sample_NN, SYNC)

DS_TYPE12_fixed = fix_DS_TYPE12(time_mask, ds_sample_NP, DS_TYPE12)

scipy.io.savemat(DS_TYPE_12_path_fixed, DS_TYPE12_fixed)

In [57]:
session = '2020-01-16_17-56-05'
DS_TYPE_12_path = PATH / 'fix_NN_timestamps' / 'DS_TYPE12' / "2020-01-16_17-55-45_L900.mat"
DS_TYPE_12_path_fixed = PATH / session / f'{session}_DS_TYPE12.mat'
SYNC_path = PATH / 'fix_NN_timestamps' / 'SYNC' / "L900.mat"

DS_TYPE12 = scipy.io.loadmat(str(DS_TYPE_12_path))
ds_sample_NN = DS_TYPE12['samplesDS'].flatten()

SYNC = scipy.io.loadmat(SYNC_path)

time_mask, ds_sample_NP = fix_NN_timestamps(ds_sample_NN, SYNC)

DS_TYPE12_fixed = fix_DS_TYPE12(time_mask, ds_sample_NP, DS_TYPE12)

scipy.io.savemat(DS_TYPE_12_path_fixed, DS_TYPE12_fixed)

In [58]:
session = '2020-01-16_20-25-54'
DS_TYPE_12_path = PATH / 'fix_NN_timestamps' / 'DS_TYPE12' / "2020-01-16_20-25-52_L400x2.mat"
DS_TYPE_12_path_fixed = PATH / session / f'{session}_DS_TYPE12.mat'
SYNC_path = PATH / 'fix_NN_timestamps' / 'SYNC' / "L400x2.mat"

DS_TYPE12 = scipy.io.loadmat(str(DS_TYPE_12_path))
ds_sample_NN = DS_TYPE12['samplesDS'].flatten()

SYNC = scipy.io.loadmat(SYNC_path)

time_mask, ds_sample_NP = fix_NN_timestamps(ds_sample_NN, SYNC)

DS_TYPE12_fixed = fix_DS_TYPE12(time_mask, ds_sample_NP, DS_TYPE12)

scipy.io.savemat(DS_TYPE_12_path_fixed, DS_TYPE12_fixed)